In [ ]:
!pip install --upgrade pip
!pip install fosforml numpy pandas matplotlib scikit-learn seaborn python-dateutil
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml 
!pip install fosforio
!pip install refractio
!pip install refractml

In [ ]:
!pip install seaborn scipy xgboost pandas dice-ml tabulate numpy scikit-learn pandas-profiling plotly matplotlib scipy statsmodels seaborn pydantic-settings

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [2]:
from refractio import snowflake
snowflake.get_connection(connection_name="Final hr attrition data")

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.
Exception occurred in getting snowflake connection: 'connectionSources'


In [3]:
df = snowflake.get_dataframe("Final_HR_Attrition_Data")
df

,SALARY,SENIORITY,TENURE_MONTHS,MONTHS_AFTER_COLLEGE,BIRTH_YEAR,AGE,MAPPED_ROLE_CLEAN,SEX,ETHNICITY,HOSPITAL_TYPE,...,OVER_TIME_HOURS,CHURN,BUSINESS_TRAVEL,ENVIRONMENT_SATISFACTION,JOB_SATISFACTION,MARTIAL_STATUS,PERCENTAGE_SALARY_HIKE,PERFORMANCE_RATING,RELATIONSHIP_SATISFACTION,WORK_LIFE_BALANCE
0,31657.073,1,13,161,1986,38,occupational,F,Black,Psychiatric,...,7,True,Travel_Rarely,High,Medium,Married,18,Excellent,Medium,Better
1,82697.324,2,10,72,1988,36,social,M,API,Acute Care - Department of Defense,...,6,False,Travel_Rarely,High,Very High,Married,19,Excellent,Low,Good
2,67291.312,1,65,7,1986,38,nurse,M,Black,Acute Care - Department of Defense,...,9,False,Non-Travel,High,Low,Married,14,Excellent,Very High,Bad
3,41186.945,1,36,33,1988,36,technologist,F,Hispanic,Psychiatric,...,8,False,Travel_Rarely,Low,Very High,Divorced,21,Low,Medium,Bad
4,54826.128,2,9,23,1990,34,physician,M,Black,Childrens,...,11,True,Travel_Rarely,High,Very High,Single,16,Good,High,Better
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,97520.177,5,45,113,1969,55,physician,M,Hispanic,Acute Care Hospitals,...,1,False,Travel_Rarely,High,High,Divorced,20,Outstanding,Low,Best
299996,55666.168,2,32,5,1987,37,emt,M,White,Childrens,...,9,True,Travel_Rarely,Medium,Low,Married,23,Low,Very High,Best
299997,49866.716,2,34,4,1986,38,physician,F,White,Childrens,...,0,True,Travel_Rarely,Medium,Medium,Single,14,Excellent,Medium,Bad
299998,77706.113,2,6,26,1996,28,physician,F,API,Childrens,...,1,False,Travel_Rarely,High,High,Single,11,Good,Very High,Good


In [4]:
print(df.isnull().sum())

SALARY                           0
SENIORITY                        0
TENURE_MONTHS                    0
MONTHS_AFTER_COLLEGE             0
BIRTH_YEAR                       0
AGE                              0
MAPPED_ROLE_CLEAN                0
SEX                              0
ETHNICITY                        0
HOSPITAL_TYPE                    0
HOSPITAL_OWNERSHIP               0
COMPANY_NAME                     0
CITY                             0
STATE                            0
DISTANCE                         0
DEGREE_CLEAN                     0
SCHOOL_END_DATE                  0
JOB_START_DATE                   0
JOB_END_DATE                 90102
USER_ID                          0
OVER_TIME_HOURS                  0
CHURN                            0
BUSINESS_TRAVEL                  0
ENVIRONMENT_SATISFACTION         0
JOB_SATISFACTION                 0
MARTIAL_STATUS                   0
PERCENTAGE_SALARY_HIKE           0
PERFORMANCE_RATING               0
RELATIONSHIP_SATISFA

In [5]:
df = df.dropna()

In [6]:
print(df.isnull().sum())

SALARY                       0
SENIORITY                    0
TENURE_MONTHS                0
MONTHS_AFTER_COLLEGE         0
BIRTH_YEAR                   0
AGE                          0
MAPPED_ROLE_CLEAN            0
SEX                          0
ETHNICITY                    0
HOSPITAL_TYPE                0
HOSPITAL_OWNERSHIP           0
COMPANY_NAME                 0
CITY                         0
STATE                        0
DISTANCE                     0
DEGREE_CLEAN                 0
SCHOOL_END_DATE              0
JOB_START_DATE               0
JOB_END_DATE                 0
USER_ID                      0
OVER_TIME_HOURS              0
CHURN                        0
BUSINESS_TRAVEL              0
ENVIRONMENT_SATISFACTION     0
JOB_SATISFACTION             0
MARTIAL_STATUS               0
PERCENTAGE_SALARY_HIKE       0
PERFORMANCE_RATING           0
RELATIONSHIP_SATISFACTION    0
WORK_LIFE_BALANCE            0
dtype: int64


In [7]:
df = df.drop(["USER_ID", "JOB_START_DATE", "JOB_END_DATE", "SCHOOL_END_DATE"], axis = 1)

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
CATEGORICAL_COLUMNS = ["MAPPED_ROLE_CLEAN","SEX", "ETHNICITY","HOSPITAL_TYPE", "HOSPITAL_OWNERSHIP","COMPANY_NAME","CITY","STATE","DISTANCE", 
                       "DEGREE_CLEAN","BUSINESS_TRAVEL","ENVIRONMENT_SATISFACTION","JOB_SATISFACTION","MARTIAL_STATUS","PERFORMANCE_RATING","RELATIONSHIP_SATISFACTION","WORK_LIFE_BALANCE"]
NUMERICAL_COLUMNS = ["SALARY", "SENIORITY", "TENURE_MONTHS", "MONTHS_AFTER_COLLEGE", "BIRTH_YEAR","AGE", "OVER_TIME_HOURS", "PERCENTAGE_SALARY_HIKE",]
LABEL_COLUMNS = ["CHURN"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [10]:
feature_columns = CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS 

In [11]:
feature_columns = [col for col in feature_columns if col in df.columns]
LABEL_COLUMNS = [col for col in LABEL_COLUMNS if col in df.columns]

In [12]:
X = df[feature_columns]
Y = df[LABEL_COLUMNS]

In [13]:
X.head()

,MAPPED_ROLE_CLEAN,SEX,ETHNICITY,HOSPITAL_TYPE,HOSPITAL_OWNERSHIP,COMPANY_NAME,CITY,STATE,DISTANCE,DEGREE_CLEAN,...,RELATIONSHIP_SATISFACTION,WORK_LIFE_BALANCE,SALARY,SENIORITY,TENURE_MONTHS,MONTHS_AFTER_COLLEGE,BIRTH_YEAR,AGE,OVER_TIME_HOURS,PERCENTAGE_SALARY_HIKE
0,occupational,F,Black,Psychiatric,Government - Local,24-7 Bright Star Healthcare LLC,Jacksonville,FL,5-10 Miles,Bachelors Degree,...,Medium,Better,31657.073,1,13,161,1986,38,7,18
1,social,M,API,Acute Care - Department of Defense,Government - Hospital District or Authority,Mayo Clinic,Deltona,FL,<2 Miles,Masters Degree,...,Low,Good,82697.324,2,10,72,1988,36,6,19
2,nurse,M,Black,Acute Care - Department of Defense,Physician,Cleveland Clinic,Reno,NV,<2 Miles,Masters Degree,...,Very High,Bad,67291.312,1,65,7,1986,38,9,14
3,technologist,F,Hispanic,Psychiatric,Government - State,24-7 Bright Star Healthcare LLC,Phoenix,AZ,5-10 Miles,Undergraduate Diploma,...,Medium,Bad,41186.945,1,36,33,1988,36,8,21
4,physician,M,Black,Childrens,Voluntary non-profit - Other,McKesson Corp.,Janesville,WI,5-10 Miles,Doctoral Degree,...,High,Better,54826.128,2,9,23,1990,34,11,16


In [14]:
Y.head()

,CHURN
0,True
1,False
2,False
3,False
4,True


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.20, random_state = 42)

In [16]:
## train_df and test_df are both snowpark dataframes
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

categorical_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
)

numerical_transformer = make_pipeline(
    SimpleImputer(strategy='mean'),
    MinMaxScaler(clip=True)
)

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, CATEGORICAL_COLUMNS),
        ('num', numerical_transformer, NUMERICAL_COLUMNS)
    ]
)


pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier())
])

pipeline.fit(X_train, y_train)

result = pipeline.predict(X_test)

In [17]:
result

array([0, 1, 1, ..., 1, 1, 1])

In [52]:
print("Prediction:", result)

Prediction: [0 1 1 ... 1 1 1]


In [18]:
from joblib import dump, load
filename = "HR_Attrition_ml_model.joblib"
dump(pipeline, filename)

['HR_Attrition_ml_model.joblib']

In [19]:
import requests

In [20]:
y_train.columns

Index(['CHURN'], dtype='object')

In [21]:
model = load(filename)

In [22]:
y_pred = pipeline.predict(X_test)
y_prob = pipeline.predict_proba(X_test)

In [53]:
print("Prediction:", y_pred)
print("Predicted probabilities:", y_prob)

Prediction: [0 1 1 ... 1 1 1]
Predicted probabilities: [[0.51492083 0.48507917]
 [0.42092532 0.5790747 ]
 [0.42017162 0.5798284 ]
 ...
 [0.489658   0.510342  ]
 [0.35479176 0.64520824]
 [0.43741506 0.56258494]]


In [23]:
from refractml import *

from refractml.constants import MLModelFlavours

/tmp/pip_packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [30]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    data_json = eval(payload)
    data = pd.DataFrame([data_json])
    prediction = str(model.predict(data)[0])
    return prediction

In [31]:
import requests
payload = str(X_test.iloc[123].to_dict())
req = requests.Request()
req.json = {"payload": payload}

print(score(pipeline, req))

1


In [33]:
req.json

{'payload': "{'MAPPED_ROLE_CLEAN': 'occupational', 'SEX': 'M', 'ETHNICITY': 'White', 'HOSPITAL_TYPE': 'Childrens', 'HOSPITAL_OWNERSHIP': 'Physician', 'COMPANY_NAME': 'Johnson & Johnson', 'CITY': 'Morgantown', 'STATE': 'WV', 'DISTANCE': '>10 Miles', 'DEGREE_CLEAN': 'Masters Degree', 'BUSINESS_TRAVEL': 'Travel_Rarely', 'ENVIRONMENT_SATISFACTION': 'Low', 'JOB_SATISFACTION': 'Low', 'MARTIAL_STATUS': 'Divorced', 'PERFORMANCE_RATING': 'Excellent', 'RELATIONSHIP_SATISFACTION': 'Medium', 'WORK_LIFE_BALANCE': 'Better', 'SALARY': 63456.001, 'SENIORITY': 3, 'TENURE_MONTHS': 14, 'MONTHS_AFTER_COLLEGE': 51, 'BIRTH_YEAR': 1989, 'AGE': 35, 'OVER_TIME_HOURS': 3, 'PERCENTAGE_SALARY_HIKE': 19}"}

In [ ]:
{"payload": "{'MAPPED_ROLE_CLEAN': 'occupational', 'SEX': 'M', 'ETHNICITY': 'White', 'HOSPITAL_TYPE': 'Childrens', 'HOSPITAL_OWNERSHIP': 'Physician', 'COMPANY_NAME': 'Johnson & Johnson', 'CITY': 'Morgantown', 'STATE': 'WV', 'DISTANCE': '>10 Miles', 'DEGREE_CLEAN': 'Masters Degree', 'BUSINESS_TRAVEL': 'Travel_Rarely', 'ENVIRONMENT_SATISFACTION': 'Low', 'JOB_SATISFACTION': 'Low', 'MARTIAL_STATUS': 'Divorced', 'PERFORMANCE_RATING': 'Excellent', 'RELATIONSHIP_SATISFACTION': 'Medium', 'WORK_LIFE_BALANCE': 'Better', 'SALARY': 63456.001, 'SENIORITY': 3, 'TENURE_MONTHS': 14, 'MONTHS_AFTER_COLLEGE': 51, 'BIRTH_YEAR': 1989, 'AGE': 35, 'OVER_TIME_HOURS': 3, 'PERCENTAGE_SALARY_HIKE': 19}"}

In [34]:
## registering the model in refract.
tmp = register_model(pipeline, 
               score, 
               name="HR_Analytic_ml_model_Final", 
               description="Final_HR_ANALYTICS_model_trained_using _ml",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="\\n pip install fosforml \\n pip install seaborn \\n pip install snowflake-connector-python[pandas] \\n pip install joblib==1.3.2 scikit-learn=1.3.2",
               y_true=y_test,
               y_pred=y_pred, 
               features=X_train.columns,
               labels=[0,1],
               input_type="json", 
               explain_ai=True,
               prob=y_prob,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               target_names=['NOT LEFT','LEFT'],
               kyd=True, kyd_score = True)


Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


In [35]:
payload  = {"payload": X_test.iloc[0].to_dict()}
payload

{'payload': {'MAPPED_ROLE_CLEAN': 'occupational',
  'SEX': 'M',
  'ETHNICITY': 'API',
  'HOSPITAL_TYPE': 'Acute Care - Department of Defense',
  'HOSPITAL_OWNERSHIP': 'Voluntary non-profit - Other',
  'COMPANY_NAME': 'Encompass Health Corp.',
  'CITY': 'Texarkana',
  'STATE': 'TX',
  'DISTANCE': '5-10 Miles',
  'DEGREE_CLEAN': 'Associates Degree',
  'BUSINESS_TRAVEL': 'Travel_Rarely',
  'ENVIRONMENT_SATISFACTION': 'Very High',
  'JOB_SATISFACTION': 'High',
  'MARTIAL_STATUS': 'Married',
  'PERFORMANCE_RATING': 'Low',
  'RELATIONSHIP_SATISFACTION': 'High',
  'WORK_LIFE_BALANCE': 'Bad',
  'SALARY': 50805.547,
  'SENIORITY': 2,
  'TENURE_MONTHS': 26,
  'MONTHS_AFTER_COLLEGE': 105,
  'BIRTH_YEAR': 1981,
  'AGE': 43,
  'OVER_TIME_HOURS': 3,
  'PERCENTAGE_SALARY_HIKE': 21}}

In [37]:
payload["payload"].keys()

dict_keys(['MAPPED_ROLE_CLEAN', 'SEX', 'ETHNICITY', 'HOSPITAL_TYPE', 'HOSPITAL_OWNERSHIP', 'COMPANY_NAME', 'CITY', 'STATE', 'DISTANCE', 'DEGREE_CLEAN', 'BUSINESS_TRAVEL', 'ENVIRONMENT_SATISFACTION', 'JOB_SATISFACTION', 'MARTIAL_STATUS', 'PERFORMANCE_RATING', 'RELATIONSHIP_SATISFACTION', 'WORK_LIFE_BALANCE', 'SALARY', 'SENIORITY', 'TENURE_MONTHS', 'MONTHS_AFTER_COLLEGE', 'BIRTH_YEAR', 'AGE', 'OVER_TIME_HOURS', 'PERCENTAGE_SALARY_HIKE'])

In [45]:
headers={"Content-type":"application/json"}
url = "http://svc-de32d652-d845-462d-8e8e-a95b2c068241:5001/hranalyticmlmodelfinal/cf744423-f68a-456b-b737-8b9dd9678361/score"
data={"payload": payload}
response = requests.post(url, json=data, headers=headers) 
response.json()

{'data': '1', 'request_id': '55f4640e-337d-4e27-afba-45ae6c20c249'}

In [54]:
result = pipeline.predict(X_test)
result_prob = pipeline.predict_proba(X_test)
pred_df = X_test.copy()
result = result
result_prob = result_prob
pred_df["PREDICTION"] = result
pred_df["PROB"] = result_prob[:, 1]

In [55]:
print(pred_df.head())

       MAPPED_ROLE_CLEAN SEX ETHNICITY                       HOSPITAL_TYPE  \
85051       occupational   M       API  Acute Care - Department of Defense   
20907       technologist   F     Black                           Childrens   
129259               emt   M     Black                Acute Care Hospitals   
137849      technologist   F       API                           Childrens   
107412      occupational   M     Black  Acute Care - Department of Defense   

                   HOSPITAL_OWNERSHIP                     COMPANY_NAME  \
85051    Voluntary non-profit - Other           Encompass Health Corp.   
20907   Voluntary non-profit - Church  24-7 Bright Star Healthcare LLC   
129259             Government - Local          National Research Corp.   
137849  Voluntary non-profit - Church                Johnson & Johnson   
107412          Department of Defense          National Research Corp.   

              CITY STATE    DISTANCE       DEGREE_CLEAN  ...      SALARY  \
85051    T

In [57]:
## scores
from sklearn.metrics import classification_report, log_loss, roc_auc_score
y_test_binary = y_test["CHURN"]
y_test_binary = [1 if i  else 0 for i in y_test["CHURN"]]
y_pred = [int(i) for i in pred_df["PREDICTION"]]
print(classification_report(y_test_binary, y_pred))

accuracy = sum([1 if y_test_binary[i] == y_pred[i] else 0 for i in range(len(y_test_binary))]) / len(y_test_binary)
print("accuracy: ", accuracy )

log_loss_value = log_loss(y_test_binary, result_prob)
roc_auc = roc_auc_score(y_test_binary, result_prob[:, 1])

TypeError: list indices must be integers or slices, not str

In [50]:
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
import seaborn as sns
ns_probs = [0 for _ in range(len(y_test))]
ns_auc = roc_auc_score(y_test, ns_probs)
lr_auc = roc_auc_score(y_test, pred_df["PROB"])
ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(y_test, pred_df["PROB"])
# plot the roc curve for the model
plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
plt.plot(lr_fpr, lr_tpr, marker='.', label='Radnomforest')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()

KeyError: 'PROB'